In [1]:
import pandas as pd
import yaml
import csv
import json
import time

import torch
import torch.nn as nn
from torch import nn, optim, Tensor
import torch.nn.functional as F

import numpy as np
from typing import List, Tuple, Any, Optional, Dict

from training.data import Instance, Data, Dictionary, triples_to_indices
from models.encoders import EncoderLSTM
from models.decoders import DecoderLSTM

import time

Triple = Fact = Tuple[str, str, str]
Story = List[Fact]

In [2]:
data = Data()
dictionary = Dictionary(data)
story, target = data.train[10000].story, data.train[10000].target
s, p, o = target
print(f'Number of entities {dictionary.num_entities}')
print(f'Example story: {story}')
print(f'Example target: {target}')

entity_embeddings = nn.Embedding(dictionary.num_entities, 100, sparse=True)
relation_embeddings = nn.Embedding(dictionary.num_relations, 100, sparse=True)
print(f'entity embeddings shape: {entity_embeddings.weight.shape}')

indices, ent, rel = triples_to_indices(dictionary, story)

Number of entities 78
Example story: [('Paul', 'mother', 'Michelle'), ('Michelle', 'brother', 'Elliott'), ('Elliott', 'sister', 'Valerie')]
Example target: ('Paul', 'aunt', 'Valerie')
entity embeddings shape: torch.Size([78, 100])


In [3]:
indices_t, ent_t, rel_t = triples_to_indices(dictionary, [target])

In [4]:
indices

tensor([[60, 57, 12],
        [57, 26,  1],
        [26, 73, 16]])

In [5]:
indices_t

tensor([[60, 73,  0]])

In [6]:
ent_t

tensor([[60, 73]])

In [7]:
encoder = EncoderLSTM(hidden_size = 200,
                    entity_embeddings = entity_embeddings,
                    relation_embeddings = relation_embeddings)

decoder = DecoderLSTM(hidden_size = 200,
                    entity_embeddings = entity_embeddings,
                    num_relations = dictionary.num_relations)

In [8]:
encoder_output = encoder.forward(indices, None)
decoder_output = decoder.forward(ent_t, encoder_output)

torch.Size([1, 2, 200])
torch.Size([2, 200])
torch.Size([2, 22])


In [10]:
decoder_output.shape

torch.Size([1, 2, 22])

In [9]:
out[0,:10]

tensor([-0.1782, -0.0239, -0.0902,  0.1378,  0.0522, -0.0274, -0.1502,  0.0907,
         0.1995, -0.0682], grad_fn=<SliceBackward>)

In [10]:
h.shape

torch.Size([1, 1, 200])

In [11]:
c.shape

torch.Size([1, 1, 200])